# Set connection with GitHub

In [1]:
# ! cd /kaggle/working && rm -rf RECsys_Challenge2024

In [2]:
from kaggle_secrets import UserSecretsClient

token = UserSecretsClient().get_secret("Token")

! git clone https://{token}@github.com/madratak/RECsys_Challenge2024.git

Cloning into 'RECsys_Challenge2024'...
remote: Enumerating objects: 4427, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (27/27), done.
remote: Total 4427 (delta 13), reused 15 (delta 4), pack-reused 4388 (from 1)
Receiving objects: 100% (4427/4427), 174.77 MiB | 29.20 MiB/s, done.
Resolving deltas: 100% (2594/2594), done.
Updating files: 100% (406/406), done.


In [3]:
! pip install PyGithub requests

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 33.1 MB/s eta 0:00:00


In [4]:
%cd /kaggle/working/RECsys_Challenge2024 
! python run_compile_all_cython.py

/kaggle/working/RECsys_Challenge2024
run_compile_all_cython: Found 11 Cython files in 5 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/usr/bin/python3'
Compiling [1/11]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarraytypes.h:1929,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1252:
/usr/local/lib/python3.10/dist-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wcpp-Wcpp]8;;]
   17 | #warning "Using deprecated NumPy API, disabl

In [5]:
from Utils.notebookFunctions import *
import numpy as np
import pandas as pd
import scipy.sparse as sps
import time
import shutil
import optuna
import json
import os
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit


K_PATH = '/kaggle/working/RECsys_Challenge2024'
GH_PATH = 'TrainedModels/WithoutKFCV/GraphBased'

np.random.seed(42)

## Import the repository

In [6]:
repo = get_repo_from_github(token)

Repository 'RECsys_Challenge2024' found.


In [7]:
config = {
    'model': 'RP3betaICM',
    'metric': 'MAP',
    'tune_parameters': True,
    'database_path': '/kaggle/working/history_RP3betaICM_MAP.db',
    'copy_prev_best_params': False,
    'tune_best_params': True,
    'save_github': True
}

Import the database where previous tuning trials have been saved.

In [8]:
try:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

# Construction of URM and ICM matrices

In [9]:
URM_all_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_train.csv")

n_users = len(URM_all_dataframe["user_id"].unique())
n_items = len(URM_all_dataframe["item_id"].unique())

URM_all = sps.csr_matrix((URM_all_dataframe["data"].values, 
                          (URM_all_dataframe["user_id"].values, URM_all_dataframe["item_id"].values)),
                        shape = (n_users, n_items))

URM_all

<35736x38121 sparse matrix of type '<class 'numpy.float64'>'
	with 1764607 stored elements in Compressed Sparse Row format>

In [10]:
ICM_dataframe = pd.read_csv("/kaggle/input/recommender-system-2024-challenge-polimi/data_ICM_metadata.csv")

n_items = len(ICM_dataframe["item_id"].unique())
n_features = len(ICM_dataframe["feature_id"].unique())

ICM_all = sps.csr_matrix((ICM_dataframe["data"].values, 
                          (ICM_dataframe["item_id"].values, ICM_dataframe["feature_id"].values)),
                        shape = (n_items, n_features))

ICM_all

<38121x94331 sparse matrix of type '<class 'numpy.float64'>'
	with 2940040 stored elements in Compressed Sparse Row format>

# Training

In [11]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

EvaluatorHoldout: Ignoring 141 ( 0.4%) Users that have less than 1 test interactions


In [12]:
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender

def objective_function_RP3beta(optuna_trial):
    
    rp3beta_icm_recommender = RP3betaRecommender(ICM_all.T)
    
    full_hyperp = {
                    "topK": optuna_trial.suggest_int("topK", 0, 1500),
                   "alpha": optuna_trial.suggest_float("alpha", 0, 3, log=False),
                   "beta": optuna_trial.suggest_float("beta", 0, 3, log=False),
                   "normalize_similarity": optuna_trial.suggest_categorical("normalize_similarity", [True, False]),
                   "implicit": optuna_trial.suggest_categorical("implicit", [True, False]),
                  }        
    
    rp3beta_icm_recommender.fit(**full_hyperp)
    
    recommender_instance = ItemKNNCustomSimilarityRecommender(URM_train)
    recommender_instance.fit(rp3beta_icm_recommender.W_sparse)
    
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [13]:
if config['tune_parameters']:
    
    optuna_study = optuna.create_study(direction='maximize', study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}',
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)

    optuna_study.optimize(objective_function_RP3beta, n_trials=200)

[I 2024-12-24 09:28:21,968] Using an existing study with name 'hyperparameters_tuning_RP3betaICM_MAP' instead of creating a new one.


RP3betaRecommender: Similarity column 38121 (100.0%), 318.21 column/sec. Elapsed time 2.00 min
EvaluatorHoldout: Processed 35595 (100.0%) in 50.53 sec. Users per second: 704


[I 2024-12-24 09:31:45,874] Trial 200 finished with value: 0.006494008878543251 and parameters: {'topK': 1253, 'alpha': 0.7353318308841013, 'beta': 1.2688758586217257, 'normalize_similarity': True, 'implicit': False}. Best is trial 94 with value: 0.023107177974434116.


RP3betaRecommender: Similarity column 38121 (100.0%), 706.86 column/sec. Elapsed time 53.93 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 34.88 sec. Users per second: 1021


[I 2024-12-24 09:33:17,155] Trial 201 finished with value: 0.02077057929038535 and parameters: {'topK': 36, 'alpha': 0.5564877850397256, 'beta': 0.32651049103587776, 'normalize_similarity': True, 'implicit': False}. Best is trial 94 with value: 0.023107177974434116.


...

[I 2024-12-24 14:36:43,710] Trial 397 finished with value: 0.01840982771345179 and parameters: {'topK': 33, 'alpha': 0.4589804699865398, 'beta': 0.34814517859585437, 'normalize_similarity': False, 'implicit': False}. Best is trial 94 with value: 0.023107177974434116.


RP3betaRecommender: Similarity column 38121 (100.0%), 673.17 column/sec. Elapsed time 56.63 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 38.70 sec. Users per second: 920


[I 2024-12-24 14:38:22,525] Trial 398 finished with value: 0.01980183368004683 and parameters: {'topK': 70, 'alpha': 0.2733349403547618, 'beta': 0.39803871263650503, 'normalize_similarity': True, 'implicit': False}. Best is trial 94 with value: 0.023107177974434116.


RP3betaRecommender: Similarity column 38121 (100.0%), 705.52 column/sec. Elapsed time 54.03 sec
EvaluatorHoldout: Processed 35595 (100.0%) in 34.35 sec. Users per second: 1036


[I 2024-12-24 14:39:52,942] Trial 399 finished with value: 0.02155078852255399 and parameters: {'topK': 29, 'alpha': 0.3549511090388391, 'beta': 0.25147500657743965, 'normalize_similarity': True, 'implicit': True}. Best is trial 94 with value: 0.023107177974434116.


## Some optuna visualizations on recommender parameters

In [14]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_slice(optuna_study)
fig.show()

In [15]:
if not config['tune_parameters']:
    optuna_study = optuna.load_study(study_name=f'hyperparameters_tuning_{config["model"]}_{config["metric"]}', storage=f'sqlite:///{config["database_path"]}')
    
fig = optuna.visualization.plot_param_importances(optuna_study)
fig.show()

## Let's train the recommender with best parameter values

In [16]:
if config['tune_best_params']:

    if config['tune_parameters']:
        best_params = optuna_study.best_trial.params
    else: 
        with open(f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 'r') as best_params_json:
            best_params = json.load(best_params_json)

    rp3beta_icm_recommender = RP3betaRecommender(ICM_all.T)
    rp3beta_icm_recommender.fit(**best_params)
    
    recommender_instance = ItemKNNCustomSimilarityRecommender(URM_train + URM_validation)
    recommender_instance.fit(rp3beta_icm_recommender.W_sparse)

RP3betaRecommender: Similarity column 38121 (100.0%), 707.46 column/sec. Elapsed time 53.88 sec


# Testing

Create the recommendations for the submission. 

In [17]:
if config['tune_best_params']:

    data_target_users_test = pd.read_csv('/kaggle/input/recommender-system-2024-challenge-polimi/data_target_users_test.csv')
    create_submission(data_target_users_test, recommender_instance, f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv')

Submission file saved as /kaggle/working/submission_RP3betaICM_MAP.csv


# Save Version on GitHub 

Write or import a json file where best hyperparameters are saved. 

In [18]:
if config['tune_parameters']:
    with open(f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 'w') as params_file:
        json.dump(optuna_study.best_params, params_file)
        
    if config['save_github']:
        upload_file(
            f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/best_params_{config["model"]}_{config["metric"]}.json', 
            f'{config["model"]}_{config["metric"]} tuning results (from kaggle notebook)',
            repo
        )
elif config['copy_prev_best_params']:
    shutil.copyfile(
        f'{K_PATH}/{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/'\
        f'best_params_{config["model"]}_{config["metric"]}.json', 
        f'/kaggle/working/best_params_{config["model"]}_{config["metric"]}.json'
    )

File 'TrainedModels/WithoutKFCV/GraphBased/RP3betaICMRecommender/OptimizingMAP/best_params_RP3betaICM_MAP.json' updated successfully.


Save the history of the tuned model.

In [19]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/history_{config["model"]}_{config["metric"]}.db',
        f'Tuning {config["model"]}_{config["metric"]} db updated results (from kaggle notebook)',
        repo
    )

File 'TrainedModels/WithoutKFCV/GraphBased/RP3betaICMRecommender/OptimizingMAP/history_RP3betaICM_MAP.db' updated successfully.


Save the best trained model and its submission.

In [20]:
if config['save_github'] and config['tune_best_params']: 
    upload_file(
                f'/kaggle/working/submission_{config["model"]}_{config["metric"]}.csv', 
                f'{GH_PATH}/{config["model"]}Recommender/Optimizing{config["metric"]}/Submission/submission_{config["model"]}_{config["metric"]}.csv', 
                f'New {config["model"]}_{config["metric"]} submission (from kaggle notebook)',
                repo
            )

File 'TrainedModels/WithoutKFCV/GraphBased/RP3betaICMRecommender/OptimizingMAP/Submission/submission_RP3betaICM_MAP.csv' updated successfully.
